## Implementing sequential GJK algorithm with distance

In [1]:
import open3d as o3d
import numpy as np
import time

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
# visualize mesh models/link_4.obj
mesh_frame1 = o3d.io.read_triangle_mesh("models/link_4.obj")
# duplicate mesh
mesh_frame2 = o3d.geometry.TriangleMesh(mesh_frame1)
# move mesh
mesh_frame2.translate((1.0, 0.0, 0.0))

TriangleMesh with 1229 points and 2454 triangles.

In [3]:
# draw a line from (0,0,0) to (1,0,0)
points = [[0, 0, 0], [1, 0, 0], [0, 1, 0], [1, 1, 0], [0, 0, 1], [1, 0, 1],
            [0, 1, 1], [1, 1, 1]]
lines = [[0, 1], [0, 2], [1, 3], [2, 3], [4, 5], [4, 6], [5, 7], [6, 7],
            [0, 4], [1, 5], [2, 6], [3, 7]]
colors = [[1, 0, 0] for i in range(len(lines))]
line_set = o3d.geometry.LineSet()
line_set.points = o3d.utility.Vector3dVector(points)
line_set.lines = o3d.utility.Vector2iVector(lines)
line_set.colors = o3d.utility.Vector3dVector(colors)
o3d.visualization.draw_geometries([mesh_frame1, mesh_frame2, line_set])
# o3d.visualization.draw_geometries([mesh_frame1, mesh_frame2, line])


In [4]:
def support_function(mesh, direction, start_index=None):
    if not start_index is None:
        # perform hill climbing
        support_index = start_index
        support_value = np.dot(np.asarray(mesh.vertices)[support_index], direction)
        while True:
            adj_vertices = list(mesh.adjacency_list[support_index])
            support_values = np.dot(np.asarray(mesh.vertices)[adj_vertices], direction)
            support_index_new = adj_vertices[np.argmax(support_values)]
            support_value_new = support_values[np.argmax(support_values)]
            if support_value_new > support_value:
                support_index = support_index_new
                support_value = support_value_new
            else:
                break
    else:
        # perform brute force search
        support_values = np.dot(np.asarray(mesh.vertices), direction)
        support_index = np.argmax(support_values)
    return support_index

def to_c_space(mesh1, mesh2, simplex_pt):
    vertex1 = np.asarray(mesh1.vertices)
    vertex2 = np.asarray(mesh2.vertices)
    return vertex1[simplex_pt[0]] - vertex2[simplex_pt[1]]

def calculate_minkowski_difference_support_function(mesh1, mesh2):
    vertex1 = np.asarray(mesh1.vertices)
    vertex2 = np.asarray(mesh2.vertices)

    directions_sphere = o3d.geometry.TriangleMesh.create_sphere(radius=1.0)
    directions = np.asarray(directions_sphere.vertices)

    vertex_minkowski = []
    for d in directions:
        support_index1 = support_function(mesh1, d)
        support_index2 = support_function(mesh2, -d)
        vertex_minkowski.append(to_c_space(mesh1, mesh2, (support_index1, support_index2)))

    vertex_minkowski = np.asarray(vertex_minkowski)

    # create a point cloud from the minkowski difference
    pcd_minkowski = o3d.geometry.PointCloud()
    pcd_minkowski.points = o3d.utility.Vector3dVector(vertex_minkowski)
    
    return pcd_minkowski


def calculate_minkowski_difference(mesh1, mesh2):
    vertex1 = np.asarray(mesh1.vertices)
    vertex2 = np.asarray(mesh2.vertices)
    
    # create a new vertex list
    vertex_minkowski = []

    # for each vertex in mesh1
    for v1 in vertex1:
        # for each vertex in mesh2
        for v2 in vertex2:
            # calculate the minkowski difference
            vertex_minkowski.append(v1 - v2)
    vertex_minkowski = np.asarray(vertex_minkowski)

    # create a point cloud from the minkowski difference
    pcd_minkowski = o3d.geometry.PointCloud()
    pcd_minkowski.points = o3d.utility.Vector3dVector(vertex_minkowski)
    return pcd_minkowski

In [40]:
print("Let's define some primitives")
# mesh_box = o3d.geometry.TriangleMesh.create_box(width=1.0,
#                                                 height=1.0,
#                                                 depth=1.0)
mesh_box = o3d.geometry.TriangleMesh.create_cylinder(radius=0.3,height=4.0)
mesh_box.compute_vertex_normals()
mesh_box.compute_adjacency_list()
mesh_box.paint_uniform_color([0.9, 0.1, 0.1])
mesh_box.translate((5, 0, 2))

# mesh_box = o3d.geometry.TriangleMesh.create_sphere(radius=1.0)
# mesh_box.translate((5, 0, 2))
# mesh_box.compute_vertex_normals()
# mesh_box.paint_uniform_color([0.1, 0.1, 0.7])

mesh_sphere = o3d.geometry.TriangleMesh.create_box(width=1.0,
                                                height=1.0,
                                                depth=1.0)
mesh_sphere.compute_vertex_normals()
mesh_sphere.paint_uniform_color([0.9, 0.1, 0.1])
mesh_sphere.translate((5, 0, -2))

# mesh_sphere = o3d.geometry.TriangleMesh.create_sphere(radius=1.0)
# mesh_sphere.translate((5, 0, -2))
# mesh_sphere.compute_vertex_normals()
# mesh_sphere.compute_adjacency_list()
# mesh_sphere.paint_uniform_color([0.1, 0.1, 0.7])


mesh_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(
    size=0.6, origin=[0, 0, 0])
print("We draw a few primitives using collection.")
o3d.visualization.draw_geometries(
    [mesh_box, mesh_sphere, mesh_frame, calculate_minkowski_difference_support_function(mesh_box, mesh_sphere)])

Let's define some primitives
We draw a few primitives using collection.


In [45]:
def solve_linear_interpolation(A, B, C):
    t_values = [(C[i] - B[i]) / (A[i] - B[i]) for i in range(3)]
    return t_values

# Example usage
A = np.array([2, 4, 6])
B = np.array([5, 7, 9])
C = np.array([3, 5, 7])

t_values = solve_linear_interpolation(A, B, C)
print(t_values)


[0.6666666666666666, 0.6666666666666666, 0.6666666666666666]


In [44]:
mesh1 = mesh_box
mesh2 = mesh_sphere
    
def origin_in_simplex(mesh1, mesh2, simplex):
    # if simplex is a point
    if len(simplex) == 1:
        return True
    # if simplex is a line
    elif len(simplex) == 2:
        # get the two points
        p1 = to_c_space(mesh1, mesh2, simplex[0])
        p2 = to_c_space(mesh1, mesh2, simplex[1])
        # get the line
        line = p2 - p1
        # get the normal
        normal = np.cross(line, p1)
        # check if the origin is in the simplex
        return np.dot(normal, p1) >= 0
    # if simplex is a triangle
    elif len(simplex) == 3:
        # get the three points
        p1 = to_c_space(mesh1, mesh2, simplex[0])
        p2 = to_c_space(mesh1, mesh2, simplex[1])
        p3 = to_c_space(mesh1, mesh2, simplex[2])
        # get the normal
        normal = np.cross(p2 - p1, p3 - p1)
        # check if the origin is in the simplex
        return np.dot(normal, p1) >= 0

def barycentric(vert, p):
    if len(vert) == 1:
        return np.array([1])
    elif len(vert) == 2:
        a, b = vert
        ab = b - a
        ap = p - a
        u = np.dot(ap, ab) / np.dot(ab, ab)
        return np.array([1 - u, u])
    elif len(vert) == 3:
        a, b, c = vert
        v0 = b - a
        v1 = c - a
        v2 = p - a;
        d00 = np.dot(v0, v0)
        d01 = np.dot(v0, v1)
        d11 = np.dot(v1, v1)
        d20 = np.dot(v2, v0)
        d21 = np.dot(v2, v1)
        denom = d00 * d11 - d01 * d01
        v = (d11 * d20 - d01 * d21) / denom
        w = (d00 * d21 - d01 * d20) / denom
        u = 1.0 - v - w
        return np.array([u, v, w])
    elif len(vert) == 4:
        a, b, c, d = vert
        vap = p - a
        vbp = p - b

        vab = b - a
        vac = c - a
        vad = d - a

        vbc = c - b
        vbd = d - b

        def scalar_triple_product(a, b, c):
            return np.dot(a, np.cross(b, c))

        va6 = scalar_triple_product(vbp, vbd, vbc)
        vb6 = scalar_triple_product(vap, vac, vad)
        vc6 = scalar_triple_product(vap, vad, vab)
        vd6 = scalar_triple_product(vap, vab, vac)

        v6 = 1 / scalar_triple_product(vab, vac, vad)
        return np.array([va6 * v6, vb6 * v6, vc6 * v6, vd6 * v6])
def simplex_origin_lambda(mesh1, mesh2, simplex, tol=1e-5):
    if len(simplex) == 1:
        return [simplex[0]], [1.0]
    elif len(simplex) == 2:
        # get the two points
        p1 = to_c_space(mesh1, mesh2, simplex[0])
        p2 = to_c_space(mesh1, mesh2, simplex[1])
        # degenerate case
        if np.linalg.norm(p1 - p2) < tol:
            return simplex_origin_lambda(mesh1, mesh2, [simplex[0]], tol)
        lmdas = barycentric([p1, p2], np.zeros(3))
        lmdas_code = lmdas > tol
        # check voronoi regions
        if all(lmdas_code):
            return simplex, lmdas
        else:
            return simplex_origin_lambda(mesh1, mesh2, [simplex[i] for i in range(2) if lmdas_code[i]], tol)
    elif len(simplex) == 3:
        # get the three points
        p1 = to_c_space(mesh1, mesh2, simplex[0])
        p2 = to_c_space(mesh1, mesh2, simplex[1])
        p3 = to_c_space(mesh1, mesh2, simplex[2])
        # degenerate case
        if np.linalg.norm(np.cross(p2 - p1, p3 - p1)) < tol:
            # compare the 3 lengths and return the longest edge
            l1 = np.linalg.norm(p1 - p2)
            l2 = np.linalg.norm(p2 - p3)
            l3 = np.linalg.norm(p3 - p1)
            if l1 >= l2 and l1 >= l3:
                return simplex_origin_lambda(mesh1, mesh2, [simplex[0], simplex[1]], tol)
            elif l2 >= l1 and l2 >= l3:
                return simplex_origin_lambda(mesh1, mesh2, [simplex[1], simplex[2]], tol)
            else:
                return simplex_origin_lambda(mesh1, mesh2, [simplex[2], simplex[0]], tol)
        # check voronoi regions
        lmdas = barycentric([p1, p2, p3], np.zeros(3))
        lmdas_code = lmdas > tol
        if all(lmdas_code):
            return simplex, lmdas
        else:
            return simplex_origin_lambda(mesh1, mesh2, [simplex[i] for i in range(3) if lmdas_code[i]], tol)
    
    elif len(simplex) == 4:
        # get the three points
        p1 = to_c_space(mesh1, mesh2, simplex[0])
        p2 = to_c_space(mesh1, mesh2, simplex[1])
        p3 = to_c_space(mesh1, mesh2, simplex[2])
        p4 = to_c_space(mesh1, mesh2, simplex[3])
        # degenerate case
        # calculate the volume of the tetrahedron
        volume = np.abs(np.dot(p4 - p1, np.cross(p2 - p1, p3 - p1))) / 6
        if volume < tol:
            # find the largest triangle
            l1 = np.linalg.norm(np.cross(p2 - p1, p3 - p1))
            l2 = np.linalg.norm(np.cross(p3 - p2, p4 - p2))
            l3 = np.linalg.norm(np.cross(p4 - p3, p1 - p3))
            l4 = np.linalg.norm(np.cross(p1 - p4, p2 - p4))
            if l1 >= l2 and l1 >= l3 and l1 >= l4:
                return simplex_origin_lambda(mesh1, mesh2, [simplex[0], simplex[1], simplex[2]], tol)
            elif l2 >= l1 and l2 >= l3 and l2 >= l4:
                return simplex_origin_lambda(mesh1, mesh2, [simplex[1], simplex[2], simplex[3]], tol)
            elif l3 >= l1 and l3 >= l2 and l3 >= l4:
                return simplex_origin_lambda(mesh1, mesh2, [simplex[2], simplex[3], simplex[0]], tol)
            else:
                return simplex_origin_lambda(mesh1, mesh2, [simplex[3], simplex[0], simplex[1]], tol)
            
        # check voronoi regions
        lmdas = barycentric([p1, p2, p3, p4], np.zeros(3))
        lmdas_code = lmdas > tol
        if all(lmdas_code):
            return simplex[1:], lmdas[1:] # leave space for new c-space point
        else:
            return simplex_origin_lambda(mesh1, mesh2, [simplex[i] for i in range(4) if lmdas_code[i]], tol)
    else:
        raise ValueError("Simplex has more than 4 points")

        

# def gjk_distance(mesh1, mesh2):
vertex1 = np.asarray(mesh1.vertices)
vertex2 = np.asarray(mesh2.vertices)

# create a simplex (list  of tuples which is (witness point 1, witness point 2), w1 -> w2)
simplex = []
s1 = support_function(mesh1, np.array([0, 0, 1]))
s2 = support_function(mesh2, -np.array([0, 0, 1]))
simplex.append((s1, s2))


line_set = o3d.geometry.LineSet()


vis = o3d.visualization.Visualizer()
vis.create_window()
vis.add_geometry(mesh_box)
vis.add_geometry(mesh_sphere)
vis.add_geometry(mesh_frame)
vis.add_geometry(line_set)

# o3d.visualization.draw_geometries(
#     [mesh_box, mesh_sphere, mesh_frame,line_set, calculate_minkowski_difference_support_function(mesh_box, mesh_sphere)] + c_space_pts_mesh)
    
for t in range(100000):
    s1 = s2 = None
    for i in range(20):
        # get the next direction
        short_simplex, short_lmda = simplex_origin_lambda(mesh1, mesh2, simplex)
        short_1 = np.sum([short_lmda[i] * vertex1[short_simplex[i][0]] for i in range(len(short_simplex))], axis=0)
        short_2 = np.sum([short_lmda[i] * vertex2[short_simplex[i][1]] for i in range(len(short_simplex))], axis=0)
        d = short_1 - short_2
        dist = np.linalg.norm(d)
        # get the support point
        s1 = support_function(mesh1, -d)
        s2 = support_function(mesh2, d)

        if (s1, s2) in short_simplex:
            print("Found a loop")
            break
        else:
            simplex = list(short_simplex)
        simplex.append((s1, s2))

    points = [short_1, short_2]
    lines = [[0, 1]]
    colors = [[1, 0, 0] for i in range(len(lines))]
    line_set.points = o3d.utility.Vector3dVector(points)
    line_set.lines = o3d.utility.Vector2iVector(lines)
    line_set.colors = o3d.utility.Vector3dVector(colors)

    print(f"distance: {np.linalg.norm(short_1 - short_2):.3f}, simplex: {simplex}, direction: {d}, short_1: {short_1}, short_2: {short_2}")


    mesh_box.rotate(np.array([
        [  0.9975021, -0.0705929,  0.0024979],
        [0.0705929,  0.9950042, -0.0705929],
        [0.0024979,  0.0705929,  0.9975021],
    ]))
    mesh_sphere.rotate(np.array([
        [  0.9975021, -0.0705929,  0.0024979],
        [0.0705929,  0.9950042, -0.0705929],
        [0.0024979,  0.0705929,  0.9975021],
    ]))
    vis.update_geometry(line_set)
    vis.update_geometry(mesh_box)
    vis.update_geometry(mesh_sphere)
    vis.poll_events()
    vis.update_renderer()

    time.sleep(0.01)



distance: 1.858, simplex: [(87, 3), (7, 1), (7, 3)], direction: [-0.93371582 -1.12784939  1.14397241], short_1: [ 4.75984664 -0.92448341  0.40835863], short_2: [ 5.69356246  0.20336599 -0.73561378]
distance: 1.722, simplex: [(87, 3), (7, 1), (7, 3)], direction: [-0.92853272 -1.09001288  0.95593129], short_1: [ 4.79426323 -0.90526074  0.22765562], short_2: [ 5.72279595  0.18475214 -0.72827566]
Found a loop
distance: 1.578, simplex: [(87, 3), (7, 1), (87, 2)], direction: [-0.77245635 -1.02524718  0.91819735], short_1: [ 4.85095955 -0.80122706  0.14904045], short_2: [ 5.6234159   0.22402012 -0.76915691]
Found a loop
distance: 1.431, simplex: [(87, 3), (7, 1), (87, 2)], direction: [-0.59827715 -0.93294572  0.9057093 ], short_1: [ 4.90326927 -0.67708088  0.09673073], short_2: [ 5.50154642  0.25586483 -0.80897857]
Found a loop
distance: 1.286, simplex: [(87, 3), (7, 1), (87, 2)], direction: [-0.43814873 -0.81361581  0.89420806], short_1: [ 4.94655382 -0.54616953  0.05344618], short_2: [ 5.38

KeyboardInterrupt: 

In [ ]:
# Implements AABB Volume Hierarchies
class AABBNode:
    def __init__(self, idx, minb, maxb) -> None:
        self.min_bound = minb
        self.max_bound = maxb

        self.left = None
        self.right = None

        self.idx = idx

    def __repr__(self) -> str:
        return f"AABBNode({self.idx}, {self.min_bound}, {self.max_bound})"

    def update_bounds_recursive(self):
        if self.idx >= 0:
            return
        else:
            self.left.update_bounds()
            self.right.update_bounds()
            self.min_bound = np.minimum(self.left.min_bound, self.right.min_bound)
            self.max_bound = np.maximum(self.left.max_bound, self.right.max_bound)

    def update_bounds(self):
        if self.idx >= 0:
            return
        else:
            if self.left:
                self.min_bound = np.minimum(self.left.min_bound, self.min_bound)
                self.max_bound = np.maximum(self.left.max_bound, self.max_bound)
            if self.right:
                self.min_bound = np.minimum(self.right.min_bound, self.min_bound)
                self.max_bound = np.maximum(self.right.max_bound, self.max_bound)

    def insert(self, node):
        if self.idx >= 0:
            # leaf node
            new_parent = self.union_bounds(node)
            new_parent.left = self
            new_parent.right = node
            return new_parent
        else:
            left_cost, right_cost, branch_cost = self.cost(node)
            if left_cost < right_cost and left_cost < branch_cost:
                if self.left is None:
                    self.left = node
                else:
                    self.left = self.left.insert(node)
                self.update_bounds()
                return self
            elif right_cost < branch_cost:
                if self.right is None:
                    self.right = node
                else:
                    self.right = self.right.insert(node)
                self.update_bounds()
                return self
            else:
                new_parent = self.union_bounds(node)
                new_parent.left = self
                new_parent.right = node
                return new_parent

    def volume(self):
        return np.prod(self.max_bound - self.min_bound)
    
    def union_bounds(self, node):
        if node is None:
            return self
        result_node = AABBNode(-1, np.minimum(self.min_bound, node.min_bound), np.maximum(self.max_bound, node.max_bound))
        return result_node

    def intersect_bounds(self, node):
        if node is None:
            return None
        result_node = AABBNode(-1, np.maximum(self.min_bound, node.min_bound), np.minimum(self.max_bound, node.max_bound))
        if np.any(result_node.max_bound < result_node.min_bound):
            return None
        return result_node


    def cost(self, node):
        left_union = node.union_bounds(self.left)
        right_union = node.union_bounds(self.right)
        branch_union = node.union_bounds(self)
        left_union_intersect_right = left_union.intersect_bounds(self.right)
        right_union_intersect_left = right_union.intersect_bounds(self.left)
        branch_intersect = node.intersect_bounds(self)

        node_cost = node.volume()
        left_union_cost = left_union.volume()
        right_union_cost = right_union.volume()
        branch_union_cost = branch_union.volume()
        left_cost = self.left.volume() if self.left else 0
        right_cost = self.right.volume() if self.right else 0
        left_union_intersect_right_cost = left_union_intersect_right.volume() if left_union_intersect_right else 0
        right_union_intersect_left_cost = right_union_intersect_left.volume() if right_union_intersect_left else 0
        branch_intersect_cost = branch_intersect.volume() if branch_intersect else 0
        
        add_left_cost = branch_union_cost - node_cost + left_union_cost - left_cost + left_union_intersect_right_cost
        add_right_cost = branch_union_cost - node_cost + right_union_cost - right_cost + right_union_intersect_left_cost
        branch_cost = branch_union_cost + branch_intersect_cost

        return add_left_cost, add_right_cost, branch_cost
        

class AABBTree:
    def __init__(self) -> None:
        self.root = None
        self.nodes = []
    
    def insert(self, bound):
        if self.root is None:
            n = AABBNode(0, bound[0], bound[1])
            self.root = n
        else:
            n = AABBNode(len(self.nodes), bound[0], bound[1])
            self.root = self.root.insert(n)
        self.nodes.append(n)

    def collide_bfs(self, bound):
        n = AABBNode(-1, bound[0], bound[1])
        queue = [self.root]
        collisions = []
        while queue:
            node = queue.pop(0)
            if node.intersect_bounds(n):
                if node.idx >= 0:
                    collisions.append(node.idx)
                else:
                    queue.append(node.left)
                    queue.append(node.right)
        return collisions

    def check_tree(self):
        queue = [self.root]
        leafs = []
        while queue:
            node = queue.pop(0)
            if node.idx >= 0:
                leafs.append(node)
            else:
                if node.left:
                    assert((node.min_bound <= node.left.min_bound).all())
                    assert((node.max_bound >= node.left.max_bound).all())
                    queue.append(node.left)
                if node.right:
                    assert((node.min_bound <= node.right.min_bound).all())
                    assert((node.max_bound >= node.right.max_bound).all())
                    queue.append(node.right)
        return leafs

\begin{align}
C(\text{add left})      &= V(p \cup x) - V(p) +
                            V(l \cup x) - V(l) +
                            V((l \cup x) \cap r) \\
C(\text{add right})     &= V(p \cup x) - V(p) +
                            V(r \cup x) - V(r) +
                            V((r \cup x) \cap l) \\
C(\text{create parent}) &= V(p \cup x) + V(p \cap x)
\end{align}

In [ ]:
rand_bounds = []
for i in range(100):
    rand_bound = [np.random.rand(3), np.random.rand(3)]
    rand_bound = [np.minimum(rand_bound[0], rand_bound[1]), np.maximum(rand_bound[0], rand_bound[1])]
    rand_bounds.append(rand_bound)

tree = AABBTree()
for rand_bound in rand_bounds:
    tree.insert(rand_bound)
    tree.root.update_bounds()
    # print(len(tree.leafs_bfs()))

check_bound = [np.min([n.min_bound for n in tree.nodes], axis=0), np.max([n.max_bound for n in tree.nodes], axis=0)]
check_bound[0] = check_bound[0] + (check_bound[1] - check_bound[0]) * 0.3
check_bound[1] = check_bound[1] - (check_bound[1] - check_bound[0]) * 0.3
coll = tree.collide_bfs(check_bound)
print(len(coll))
alltest = AABBNode(-1, check_bound[0], check_bound[1])
assert(all([tree.nodes[i].intersect_bounds(alltest) is not None for i in coll]))
assert(all([(alltest.intersect_bounds(tree.nodes[i]) is None) for i in range(len(tree.nodes)) if i not in coll]))

65
